In [88]:
from idCrawler import getUserProfile
from pymongo import MongoClient
from idCrawler import getTweets
import json
from urllib.request import urlopen

In [67]:
def patientsReader_bipolar(file_name,keywords=["borderline","bpd"]):
    f = open(file_name, 'r')
    patient_ids = {}
    valid_conditions = ['o','p','c']
    for line in f:
        try:
            segments = line.strip().split("\t")
            condition = segments[0].lower()
            description = segments[1].lower()
            screen_name = segments[2]
            user_id = segments[3]
            
        except:
            print(line)
            return patient_ids
        if condition in valid_conditions:
            if any(keyword in description for keyword in keywords):
                patient_ids[user_id] = 'c'
            else:
                patient_ids[user_id] = 'o'
       
        
    return patient_ids

In [92]:
def patientsReader_BPD(file_name, keywords = ["bipolar"]):
    f = open(file_name, 'r')
    patient_ids = {}
    for line in f:
        try:
            segments = line.strip().split("\t")
            condition = segments[0]
            description = segments[1].lower()
            user_id = segments[2]
            
        except:
            print(line)
            return patient_ids
        if condition == 'o' or condition == 'O' or condition == 'p':
            if any(keyword in description for keyword in keywords):
                patient_ids[user_id] = 'c'
            else:
                patient_ids[user_id] = 'o'
            
        
    return patient_ids
def writeUsers(tweets, dbName, collectionName):
    collection = MongoClient("localhost", 27017)[dbName][collectionName]
    collection.insert(tweets)
    print("{} tweets inserted".format(len(tweets)))
    
def sentiment_analyize(tweets):
    payload = {"data": [],"language": "en"}
    
    for tweet in tweets:
        payload["data"].append({"text": tweet["text"],"id": tweet["id"]})
    
    payload = json.dumps(payload).encode('utf-8')
    response = urlopen('http://www.sentiment140.com/api/bulkClassifyJson?appid=ccha97u@gmail.com', payload) # request to server
    results = response.read().decode('"ISO-8859-1"') # get the response
    results = json.loads(results)['data']
    for i,result in enumerate(results):
       
        if result['polarity'] == 0:
            tweets[i]["polarity"] = "negative"
        elif result['polarity'] == 2:
            tweets[i]["polarity"] = "neutral"
        elif result['polarity'] == 4:
            tweets[i]["polarity"] = "positive"
        else:
             tweets[i]["polarity"] = "unknown"
    return tweets


def scan_tweets(tweets, bulk_size=10000):

    buffer = []
    sentiment_tweets = []
    for tweet in tweets:
        buffer.append(tweet)
        if len(buffer) >= bulk_size:
            print("10000 tweets  processed")
            sentiment_tweets += sentiment_analyize(buffer)
            buffer = []
    
    
    sentiment_tweets += sentiment_analyize(buffer)
    
    return sentiment_tweets

In [39]:
file_location = "tagged_bpd_descriptions.tsv"
BPD_ids = patientsReader_BPD(file_location)
comorbidities_BPD = []
for BPD_id, condition in BPD_ids.items():
    if condition == 'c':
        comorbidities_BPD.append(BPD_id)
print(len(comorbidities_BPD))

78


In [70]:
mix = []
file_location = "bipolar_profile_700_1400.csv"
bipolar_ids = patientsReader_bipolar(file_location)
comorbidities_bipolar = []
for bipolar_id, condition in bipolar_ids.items():
    if condition == 'c':
        mix.append(bipolar_id)


477


In [71]:
bipolar_ids

{'10006872': 'o',
 '100894411': 'o',
 '1014282277': 'o',
 '101816365': 'o',
 '101893572': 'o',
 '1022694266': 'o',
 '1026760129': 'o',
 '102816119': 'o',
 '1033700874': 'o',
 '103754648': 'o',
 '106416595': 'c',
 '107041924': 'o',
 '107584252': 'o',
 '1076709452': 'o',
 '1129170108': 'o',
 '1140014239': 'o',
 '114022502': 'o',
 '114352146': 'o',
 '1145823332': 'o',
 '1146285583': 'o',
 '1178821596': 'o',
 '1183389140': 'o',
 '120946668': 'o',
 '1220265109': 'o',
 '1226140512': 'o',
 '1229003286': 'o',
 '123309478': 'o',
 '1239737869': 'o',
 '1247935578': 'c',
 '1249961412': 'o',
 '1267642878': 'o',
 '1271233922': 'o',
 '1291373377': 'o',
 '1294822856': 'o',
 '1311593233': 'o',
 '1318763677': 'o',
 '132750810': 'o',
 '1330340192': 'o',
 '1330363028': 'o',
 '1350939626': 'o',
 '1357309405': 'o',
 '1388135443': 'o',
 '139835130': 'o',
 '14078363': 'o',
 '14243519': 'c',
 '1436637847': 'o',
 '1440806754': 'o',
 '1459083476': 'o',
 '147393139': 'o',
 '148718702': 'o',
 '148748179': 'o',
 '1

In [54]:
BPD_clean = {}
comorbidity_tweets = {}
lost_ids = set()

BPD_cursor = MongoClient("localhost", 27017)['idea']["BPD_581_emotion"].find()
for tweet in BPD_cursor:
    tweet_id = tweet["id"]
    user_id = str(tweet["user"]["id"])
    if user_id in BPD_ids:
        condition = BPD_ids[user_id]
        if condition == 'c':
            comorbidity_tweets[tweet_id] = tweet
        else:
            BPD_clean[tweet_id] = tweet
    else:
        lost_ids.add(user_id)
        


    
print(len(comorbidity_tweets))
print(len(BPD_clean))

58439
303818


In [73]:
bipolar_clean = {}
mix = {}
existing_ids = set()
bipolar_cursor = MongoClient("localhost", 27017)['patients']["bipolar_clean"].find()
BPD_cursor = MongoClient("localhost", 27017)['patients']["BPD_clean"].find()
mix_cursor = MongoClient("localhost", 27017)['patients']["mix_clean"].find()

cursors = [bipolar_cursor, BPD_cursor, mix_cursor]
for cursor in cursors:
    for tweet in cursor:
        tweet_id = tweet["id"]
        existing_ids.add(tweet_id)











In [74]:
len(existing_ids)

582947

In [78]:
bipolar_tweets = []
mix_tweets = []
for user_id, condition in bipolar_ids.items():
    if condition == 'o':
        bipolar_tweets += getTweets(user_id=user_id)
    else:
        mix_tweets += getTweets(user_id=user_id)

Fall asleep
Fall asleep
Fall asleep

/home/gb/research/Mental_Disorder/1_data_collecting/idCrawler.py:32: UserWarning: Bad Authentication
  warn("Bad Authentication")
/home/gb/research/Mental_Disorder/1_data_collecting/idCrawler.py:35: UserWarning: 404 not found
  warn("404 not found")



Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep
Fall asleep


/home/gb/research/Mental_Disorder/1_data_collecting/idCrawler.py:28: UserWarning: Fall asleep
  warn("Fall asleep")


In [89]:
polarity_bipolar_tweets = scan_tweets(bipolar_tweets)
polarity_mix_tweets = scan_tweets(mix_tweets)

10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed
10000 tweets  processed


In [91]:
len(polarity_mix_tweets)

14227

In [95]:
writeUsers(polarity_bipolar_tweets,"idea","bipolar_polarity")

309130 tweets inserted


In [56]:
overlap_n = 0
for user_id in bipolar_ids:
    if any(user_id == x for x in BPD_ids):
        overlap_n += 1
print(overlap_n)

61


In [53]:
len(lost_ids)

0

In [17]:
len(BPD_patietn_ids)

496

In [ ]:
keywords = ["bipolar"]
for profile in BPD_profiles:
    description = BPD_profiles["description"].lower()
    if keywords in description:
        BPD_profiles["condition"] = ["BPD","bipolar"]
    else:
        BPD_profiles["condition"] = ["BPD"]

In [ ]:
bipolar_patietn_ids = set()
bipolar_cursor = MongoClient("localhost", 27017)['idea']["bipolar_emotion"].find()
for tweet in bipolar_cursor:
    user_id = tweet["user"]["id"]
    bipolar_patietn_ids.add(user_id)
bipolar_profiles = []
for user_id in bipolar_patietn_ids:
    bipolar_profiles += getUserProfile(user_id=user_id)

    

In [ ]:
keywords = ["borderline","bpd"]
for profile in bipolar_profiles:
    description =bipolarD_profiles["description"].lower()
    if keywords in description:
        bipolar_profiles["condition"] = ["BPD","bipolar"]
    else:
        bipolar_profiles["condition"] = ["BPD"]

In [79]:
4

4